In [1]:
import nibabel as nib
from totalsegmentator.python_api import totalsegmentator
import os
import SimpleITK as sitk
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
from image_utils import convert_series_to_nifti, quantize_maps
%matplotlib widget
import gui
import registration_gui as rgui

In [ ]:
# Define directories
input_folder = r'D:\CTH_archive\CTH_NIFTI'
stripped_folder = r'D:\CTH_archive\CTH_STRIPPED_MASK'
output_folder = r'D:\CTH_archive\CTH_STRIPPED'

# Ensure the stripped and output directories exist
os.makedirs(stripped_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Skull stripping process
for filename in os.listdir(input_folder):
    if filename.endswith(('.nii')):
        patient_name = filename.split(' ')[0]  # Extract patient name
        input_path = os.path.join(input_folder, filename)
        stripped_path = os.path.join(stripped_folder, patient_name)

        # Check if the stripped file already exists
        if not os.path.exists(stripped_path):
            try:
                input_img = nib.load(input_path)
                totalsegmentator(input_img, stripped_path, roi_subset=['brain'], ml=True)
                print(f"Processed and saved: {stripped_path}")
            except Exception as e:
                print(f"Error processing {input_path}: {e}")
        else:
            print(f"Stripped file already exists, skipping: {stripped_path}")

# Applying masks to stripped images
file_counter = 0  # Initialize file counter
missing_files = []

for mask_filename in tqdm(os.listdir(stripped_folder)):
    base_filename = os.path.splitext(mask_filename)[0].rsplit('.', 1)[0]  # Handle potential double extensions
    original_path = None

    # Find matching original file
    for original_filename in os.listdir(input_folder):
        if os.path.splitext(original_filename)[0].rsplit('.', 1)[0] == base_filename:
            original_path = os.path.join(input_folder, original_filename)
            break

    if original_path:
        mask_path = os.path.join(stripped_folder, mask_filename)
        #output_filename = base_filename + f"_{file_counter:03d}_0000.nii.gz"
        output_filename = base_filename
        output_path = os.path.join(output_folder, output_filename)

        # Check if the output file already exists
        if not os.path.exists(output_path):
            print(f"Applying mask to {original_path} using {mask_path}")

            # Load and apply mask
            mask_nii = nib.load(mask_path)
            original_nii = nib.load(original_path)
            masked_data = np.where(mask_nii.get_fdata() > 0, original_nii.get_fdata(), 0)
            masked_nii = nib.Nifti1Image(masked_data, affine=original_nii.affine)

            nib.save(masked_nii, output_path)
            file_counter += 1
        else:
            print(f"Masked file already exists, skipping: {output_path}")
    else:
        print(f"No matching file found for {mask_filename} in {input_folder}")
        missing_files.append((mask_filename, input_folder))

print("Mask application complete.")
if missing_files:
    print("Missing files:", missing_files)


In [ ]:
TMAX_SOURCE_DIR = r'D:\\CTH_archive\\TMAX_NIFTI\\'
TMAX_QUNTIZED_DIR = r'D:\\CTH_archive\\TMAX_NIFTI_QUANT'

quantization_levels = 5

quantize_maps(TMAX_SOURCE_DIR, TMAX_QUNTIZED_DIR, quantization_levels=quantization_levels)

In [3]:
transforms_dir = "D:/CTH_archive/TRANSFORMS_STRIPPED"
if not os.path.exists(transforms_dir):
    os.makedirs(transforms_dir)

fixed_images_dir = "D:/CTH_archive/CTH_STRIPPED"  
moving_images_dir = "D:/CTH_archive/CTP_STRIPPED" 

def register_images(fixed_image_path, moving_image_path, transforms_dir):
    # Extract patient identifier from the file name, ensuring .nii is not included
    patient = os.path.splitext(os.path.basename(moving_image_path))[0]
    patient = os.path.splitext(patient)[0]  # Remove .nii if present

    # Construct the transform file path
    transform_file = os.path.join(transforms_dir, f'{patient}.h5')

    # Check if the transform file already exists and skip registration if it does
    if os.path.exists(transform_file):
        print(f"Transform file already exists for patient {patient}, skipping registration.")
        return

    # Load the fixed and moving images
    fixed_image = sitk.ReadImage(fixed_image_path)
    moving_image = sitk.ReadImage(moving_image_path)

    # Initialize the registration method
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingPercentage(0.05)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetOptimizerAsGradientDescentLineSearch(learningRate=0.5, numberOfIterations=200)
    registration_method.SetOptimizerScalesFromPhysicalShift()
    #registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[8, 4, 2])
    #registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[4, 2, 1])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Initialize the transform
    initial_transform = sitk.CenteredTransformInitializer(sitk.Cast(fixed_image, moving_image.GetPixelID()), 
                                                          moving_image, 
                                                          sitk.AffineTransform(fixed_image.GetDimension()),
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    registration_method.SetInitialTransform(initial_transform, True)

    try:
        # Execute the registration
        final_transform = registration_method.Execute(fixed_image, moving_image)
        
        # Save the transform
        sitk.WriteTransform(final_transform, transform_file)
        sitk.WriteImage(sitk.Resample(moving_image, fixed_image, final_transform,  sitk.sitkNearestNeighbor), f"D:/CTH_archive/CTP_STRIPPED_REG/{patient}.nii")
        print("Final metric value: {0}".format(registration_method.GetMetricValue()))
        print(
            "Optimizer's stopping condition, {0}".format(
                registration_method.GetOptimizerStopConditionDescription()
            )
        )
        print(f"Registration successful for patient: {patient}. Transform saved to {transform_file}")
    except RuntimeError as e:
        print(f"Registration failed for patient {patient}: {e}")


for filename in tqdm(os.listdir(fixed_images_dir)):
    fixed_image_path = os.path.join(fixed_images_dir, filename)
    moving_image_path = os.path.join(moving_images_dir, filename)

    if os.path.isfile(fixed_image_path) and os.path.isfile(moving_image_path):
        register_images(fixed_image_path, moving_image_path, transforms_dir)


  0%|          | 0/102 [00:00<?, ?it/s]

Final metric value: -0.24667615149076697
Optimizer's stopping condition, GradientDescentLineSearchOptimizerv4Template: Convergence checker passed at iteration 20.
Registration successful for patient: ALFORD_BARBARA. Transform saved to D:/CTH_archive/TRANSFORMS_STRIPPED\ALFORD_BARBARA.h5
Final metric value: -0.32760509814290395
Optimizer's stopping condition, GradientDescentLineSearchOptimizerv4Template: Convergence checker passed at iteration 28.
Registration successful for patient: ALLAH_MAJUSTICE. Transform saved to D:/CTH_archive/TRANSFORMS_STRIPPED\ALLAH_MAJUSTICE.h5
Final metric value: -0.19979278403478723
Optimizer's stopping condition, GradientDescentLineSearchOptimizerv4Template: Convergence checker passed at iteration 68.
Registration successful for patient: BATTLE_MARIA. Transform saved to D:/CTH_archive/TRANSFORMS_STRIPPED\BATTLE_MARIA.h5
Final metric value: -0.36338214178141753
Optimizer's stopping condition, GradientDescentLineSearchOptimizerv4Template: Convergence checker

KeyboardInterrupt: 

In [2]:
import itk
import nibabel as nib
from totalsegmentator.python_api import totalsegmentator
import os
import SimpleITK as sitk
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
from image_utils import convert_series_to_nifti, quantize_maps
%matplotlib widget

import gui
import registration_gui as rgui

transforms_dir = "D:/CTH_archive/TRANSFORMS_STRIPPED"

if not os.path.exists(transforms_dir):
    os.makedirs(transforms_dir)

fixed_images_dir = "D:/CTH_archive/CTH_STRIPPED"  
moving_images_dir = "D:/CTH_archive/CTP_STRIPPED" 

def register_images(fixed_image_path, moving_image_path, transforms_dir):
    # Extract patient identifier from the file name, ensuring .nii is not included
    patient = os.path.splitext(os.path.basename(moving_image_path))[0]
    patient = os.path.splitext(patient)[0]  # Remove .nii if present

    # Construct the transform file path
    transform_file = os.path.join(transforms_dir, f'{patient}.h5')

    # Check if the transform file already exists and skip registration if it does
    if os.path.exists(transform_file):
        print(f"Transform file already exists for patient {patient}, skipping registration.")
        return
    # Load the fixed and moving images
    fixed_image = sitk.ReadImage(fixed_image_path)
    moving_image = sitk.ReadImage(moving_image_path)

    # Perform registration
    #resultImage = sitk.Elastix(fixed_image, moving_image)
    registered_image, params = itk.elastix_registration_method(fixed_image, moving_image)

    # Create a checkerboard image
    checkerboard = sitk.CheckerBoard(fixed_image, registered_image)

    # Convert the checkerboard image to a numpy array
    checkerboard_np = sitk.GetArrayFromImage(checkerboard)

    # Display the checkerboard image
    plt.imshow(checkerboard_np, cmap='gray')
    plt.show()
    


for filename in tqdm(os.listdir(fixed_images_dir)):
    fixed_image_path = os.path.join(fixed_images_dir, filename)
    moving_image_path = os.path.join(moving_images_dir, filename)

    if os.path.isfile(fixed_image_path) and os.path.isfile(moving_image_path):
        register_images(fixed_image_path, moving_image_path, transforms_dir)

  0%|          | 0/102 [00:00<?, ?it/s]

Transform file already exists for patient ALFORD_BARBARA, skipping registration.
Transform file already exists for patient ALLAH_MAJUSTICE, skipping registration.
Transform file already exists for patient BATTLE_MARIA, skipping registration.
Transform file already exists for patient BAUM_ROBERT, skipping registration.
Transform file already exists for patient BILLIPS_JAMES, skipping registration.
Transform file already exists for patient BOGER_DAVID_S, skipping registration.
Transform file already exists for patient BROWN_ANTHONY, skipping registration.
Transform file already exists for patient CAMPAGNA_HARRY_D, skipping registration.
Transform file already exists for patient CANIGLIA_ROBERT, skipping registration.


AttributeError: module 'itk' has no attribute 'elastix_registration_method'

In [ ]:
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np

transforms_dir = "D:/CTH_archive/TRANSFORMS_STRIPPED"
if not os.path.exists(transforms_dir):
    os.makedirs(transforms_dir)

fixed_images_dir = "D:/CTH_archive/CTH_STRIPPED"  
moving_images_dir = "D:/CTH_archive/CTP_STRIPPED" 

def register_images(fixed_image_path, moving_image_path, transforms_dir):
    patient = os.path.splitext(os.path.basename(moving_image_path))[0]
    patient = os.path.splitext(patient)[0]

    transform_file = os.path.join(transforms_dir, f'{patient}.h5')

    if os.path.exists(transform_file):
        print(f"Transform file already exists for patient {patient}, skipping registration.")
        return

    fixed_image = sitk.ReadImage(fixed_image_path)
    moving_image = sitk.ReadImage(moving_image_path)

    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingPercentage(0.6)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetOptimizerAsGradientDescentLineSearch(learningRate=0.5, numberOfIterations=200)
    registration_method.SetOptimizerScalesFromPhysicalShift()
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[8, 4, 2])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[4, 2, 1])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    initial_transform = sitk.CenteredTransformInitializer(sitk.Cast(fixed_image, moving_image.GetPixelID()), 
                                                          moving_image, 
                                                          sitk.AffineTransform(fixed_image.GetDimension()),
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    registration_method.SetInitialTransform(initial_transform, True)

    try:
        final_transform = registration_method.Execute(fixed_image, moving_image)
        sitk.WriteTransform(final_transform, transform_file)
        print(f"Registration successful for patient: {patient}. Transform saved to {transform_file}")

        # Visualize and save the registration results
        visualize_and_save_registration(fixed_image, moving_image, final_transform, patient)
    except RuntimeError as e:
        print(f"Registration failed for patient {patient}: {e}")

def visualize_and_save_registration(fixed_image, moving_image, transform, patient_id):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed_image)
    resampler.SetTransform(transform)
    resampler.SetInterpolator(sitk.sitkLinear)
    registered_image = resampler.Execute(moving_image)

    fixed_image_array = sitk.GetArrayFromImage(fixed_image)
    registered_image_array = sitk.GetArrayFromImage(registered_image)

    slice_indices = [fixed_image_array.shape[0] // 4, fixed_image_array.shape[0] // 2]  # Save quarter and middle slices

    for idx, slice_idx in enumerate(slice_indices):
        fixed_slice = fixed_image_array[slice_idx, :, :]
        registered_slice = registered_image_array[slice_idx, :, :]

        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(fixed_slice, cmap='gray')
        ax.imshow(registered_slice, cmap='hot', alpha=0.5)  # Overlay the registered image
        ax.axis('off')
        plt.savefig(f'D:/CTH_archive/Registration_Visualization/{patient_id}_slice_{idx}.png', bbox_inches='tight')
        plt.close(fig)  # Ensure the figure is closed and not displayed


# Process each image pair
for filename in os.listdir(fixed_images_dir):
    fixed_image_path = os.path.join(fixed_images_dir, filename)
    moving_image_path = os.path.join(moving_images_dir, filename)

    if os.path.isfile(fixed_image_path) and os.path.isfile(moving_image_path):
        register_images(fixed_image_path, moving_image_path, transforms_dir)



In [ ]:
def resample_image(moving_image, fixed_image, ctp_image):
    desired_size = [fixed_image.GetSize()[0], fixed_image.GetSize()[1], ctp_image.GetSize()[2]]
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(moving_image)
    resampler.SetSize(desired_size)
    resampler.SetOutputSpacing([moving_image.GetSpacing()[i] * (moving_image.GetSize()[i] / desired_size[i]) for i in range(3)])
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkLinear)
    resized_moving_image = resampler.Execute(moving_image)
    resized_moving_image.SetSpacing(ctp_image.GetSpacing())
    resized_moving_image.SetOrigin(ctp_image.GetOrigin())
    resized_moving_image.SetDirection(ctp_image.GetDirection())
    return resized_moving_image

def apply_final_transform(resized_moving_image, fixed_image, transform_file_path):
    final_transform = sitk.ReadTransform(transform_file_path)
    resampled_image = sitk.Resample(resized_moving_image, 
                                    fixed_image, 
                                    final_transform, 
                                    sitk.sitkLinear, 
                                    0.0, 
                                    fixed_image.GetPixelID())
    return resampled_image

In [26]:
import SimpleITK as sitk

# Paths to your NIfTI files
files = [
    "D:\\CTH_archive\\CTP_STRIPPED_REG\\BATTLE_MARIA.nii",
    "D:\\CTH_archive\\CTP_STRIPPED\\BATTLE_MARIA.nii",
    "D:\\CTH_archive\\TMAX_NIFTI_QUANT\\REGISTERED\\BATTLE_MARIA.nii",
    "D:\\CTH_archive\\CTH_STRIPPED\\BATTLE_MARIA.nii"
]

# Loop through each file and print out the relevant information
for file_path in files:
    try:
        # Read the image file
        image = sitk.ReadImage(file_path)
        
        # Print out relevant information
        print(f"File: {file_path}")
        print(f"Dimensions: {image.GetDimension()}")
        print(f"Size: {image.GetSize()}")
        print(f"Spacing: {image.GetSpacing()}")
        print(f"Origin: {image.GetOrigin()}")
        print(f"Direction: {image.GetDirection()}")
        print(f"Pixel Type: {image.GetPixelIDTypeAsString()}")
        print(f"Number of Components per Pixel: {image.GetNumberOfComponentsPerPixel()}")
        print("-" * 50)
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")


File: D:\CTH_archive\CTP_STRIPPED_REG\BATTLE_MARIA.nii
Dimensions: 3
Size: (512, 680, 31)
Spacing: (0.302734375, 0.30294114351272583, 4.999999046325684)
Origin: (-68.8271484375, -338.61474609375, 1256.77099609375)
Direction: (0.9968748471280338, -0.07146105755660387, 0.03367277710381495, 0.056727485700367196, 0.9442108032441664, 0.3244193678856795, -0.05497755477987119, -0.32149537522318694, 0.9453138197565785)
Pixel Type: 64-bit float
Number of Components per Pixel: 1
--------------------------------------------------
File: D:\CTH_archive\CTP_STRIPPED\BATTLE_MARIA.nii
Dimensions: 3
Size: (512, 512, 149)
Spacing: (0.390625, 0.390625, 1.0)
Origin: (-99.8046875, -328.8046875, 143.8000030517578)
Direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
Pixel Type: 64-bit float
Number of Components per Pixel: 1
--------------------------------------------------
File: D:\CTH_archive\TMAX_NIFTI_QUANT\REGISTERED\BATTLE_MARIA.nii
Dimensions: 3
Size: (512, 680, 31)
Spacing: (0.302734375, 0.30294

In [27]:
import SimpleITK as sitk
import logging

def resample_image(moving_image, fixed_image, ctp_image):
    desired_size = [512, 512, ctp_image.GetSize()[2]]
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(moving_image)
    resampler.SetSize(desired_size)
    resampler.SetOutputSpacing([moving_image.GetSpacing()[i] * (moving_image.GetSize()[i] / desired_size[i]) for i in range(3)])
    #resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkLinear)
    resized_moving_image = resampler.Execute(moving_image)
    resized_moving_image.SetSpacing(ctp_image.GetSpacing())
    resized_moving_image.SetOrigin(ctp_image.GetOrigin())
    resized_moving_image.SetDirection(ctp_image.GetDirection())
    return resized_moving_image

def apply_final_transform(resized_moving_image, fixed_image, transform_file_path):
    final_transform = sitk.ReadTransform(transform_file_path)
    resampled_image = sitk.Resample(resized_moving_image, 
                                    fixed_image, 
                                    final_transform, 
                                    sitk.sitkNearestNeighbor, 
                                    0.0, 
                                    fixed_image.GetPixelID())
    return resampled_image

def apply_final_transform(resized_moving_image, fixed_image, transform_file_path):
    try:
        final_transform = sitk.ReadTransform(transform_file_path)
        
        # Ensure the resized_moving_image is valid before proceeding
        if resized_moving_image:
            resampled_image = sitk.Resample(resized_moving_image, 
                                            fixed_image, 
                                            final_transform, 
                                            sitk.sitkLinear, 
                                            0.0, 
                                            fixed_image.GetPixelID())
            return resampled_image
        else:
            logging.error("Resized moving image is invalid. Cannot apply final transform.")
            return None
    except Exception as e:
        logging.error("Failed to apply final transform: " + str(e))
        return None





# Directories
tmax_nifti_dir = r"D:/CTH_archive/TMAX_NIFTI_QUANT"
transforms_dir = r"D:/CTH_archive/TRANSFORMS_STRIPPED"
cth_stripped_dir = r"D:/CTH_archive/CTH_STRIPPED"
ctp_stripped_dir = r"D:/CTH_archive/CTP_STRIPPED"
output_dir = os.path.join(tmax_nifti_dir, "REGISTERED")

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over the transformation files
for transform_file in os.listdir(transforms_dir):
    transform_path = os.path.join(transforms_dir, transform_file)
    base_filename = transform_file.replace('.h5', '')

    # Paths to the moving, fixed, and reference (ctp) images
    moving_image_path = os.path.join(tmax_nifti_dir, base_filename + '.nii')
    fixed_image_path = os.path.join(cth_stripped_dir, base_filename + '.nii')
    ctp_image_path = os.path.join(ctp_stripped_dir, base_filename + '.nii')
    print(f"Processing {base_filename}...")

    # Check if all required files exist
    if os.path.exists(moving_image_path) and os.path.exists(fixed_image_path) and os.path.exists(ctp_image_path):
        # Load images
        moving_image = sitk.ReadImage(moving_image_path)
        fixed_image = sitk.ReadImage(fixed_image_path)
        ctp_image = sitk.ReadImage(ctp_image_path)

        # Resample the moving image
        resized_moving_image = resample_image(moving_image, fixed_image, ctp_image)

        # Apply the final transformation
        resampled_image = apply_final_transform(resized_moving_image, fixed_image, transform_path)

        # Save the resampled image
        resampled_image_path = os.path.join(output_dir, base_filename + '.nii')
        sitk.WriteImage(resampled_image, resampled_image_path)
        print(f"Processed and saved: {resampled_image_path}")
    else:
        print(f"Required files for {base_filename} are not available.")
    

Processing ALFORD_BARBARA...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\ALFORD_BARBARA.nii
Processing ALLAH_MAJUSTICE...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\ALLAH_MAJUSTICE.nii
Processing BATTLE_MARIA...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\BATTLE_MARIA.nii
Processing BAUM_ROBERT...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\BAUM_ROBERT.nii
Processing BILLIPS_JAMES...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\BILLIPS_JAMES.nii
Processing BOGER_DAVID_S...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\BOGER_DAVID_S.nii
Processing BROWN_ANTHONY...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\BROWN_ANTHONY.nii
Processing CAMPAGNA_HARRY_D...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\CAMPAGNA_HARRY_D.nii
Processing CANIGLIA_ROBERT...
Processed and saved: D:/CTH_archive/TMAX_NIFTI_QUANT\REGISTERED\CANIGLIA_ROBERT.nii


In [ ]:
import os
import re

# Directory containing the files
directory_path = "D:\\CTH_archive\\CTP_STRIPPED"

# Iterate over all files in the directory
for filename in os.listdir(directory_path):
    # Full path of the current file
    original_path = os.path.join(directory_path, filename)

    # Skip directories, process only files
    if os.path.isfile(original_path):
        # Remove numeric pattern and the word 'transform' from the filename
        new_filename = re.sub(r"_\d+|_transform", "", filename)

        # Full path for the new file name
        new_path = os.path.join(directory_path, new_filename)

        # Rename the file
        os.rename(original_path, new_path)
        print(f"Renamed '{original_path}' to '{new_path}'")


In [ ]:
def adjust_label_intensities(label_dir, output_dir):
    # Ensure the output directory exists, create if it doesn't
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over all files in the label directory
    for file_name in os.listdir(label_dir):
        if file_name.endswith('.nii'):
            file_path = os.path.join(label_dir, file_name)

            # Load the label file
            label_image = nib.load(file_path)
            label_data = label_image.get_fdata()

            # Modify non-zero intensities
            non_background_mask = label_data > 0
            label_data[non_background_mask] = np.round(label_data[non_background_mask])

            # Create a new NIfTI image object with the adjusted data
            adjusted_label_image = nib.Nifti1Image(label_data, affine=label_image.affine, header=label_image.header)

            # Define the output file path
            output_file_path = os.path.join(output_dir, file_name)

            # Save the adjusted data to the new file in the output directory
            nib.save(adjusted_label_image, output_file_path)
            print(f"Adjusted and saved: {output_file_path}")

# Directory containing the quantized label files
label_dir = r"D:\CTH_archive\TMAX_NIFTI_QUANT\REGISTERED"

# New output directory for adjusted label files
output_dir = r"D:\CTH_archive\TMAX_NIFTI_QUANT\REGISTERED_ADJUSTED"

# Call the function to adjust label intensities and save to the new directory
adjust_label_intensities(label_dir, output_dir)


In [ ]:
import os

# List of directories to check
directories = [
    "D:\\CTH_archive\\CTH_NIFTI",
    "D:\\CTH_archive\\CTH_STRIPPED",
    "D:\\CTH_archive\\CTH_STRIPPED_MASK",
    "D:\\CTH_archive\\CTP_NIFTI",
    "D:\\CTH_archive\\CTP_STRIPPED",
    "D:\\CTH_archive\\CTP_STRIPPED_MASK",
    "D:\\CTH_archive\\TMAX_NIFTI",
    "D:\\CTH_archive\\TMAX_NIFTI_QUANT"
]

# Dictionary to hold the set of file base names (without extension) for each directory
file_sets = {}

# Extract and store the base names of the files in each directory
for directory in directories:
    file_names = [os.path.splitext(f)[0] for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    file_sets[directory] = set(file_names)

# Use the first directory as the reference for comparison
reference_files = list(file_sets.values())[0]
reference_dir = directories[0]

# Compare the file sets of each directory against the reference
for directory, file_set in file_sets.items():
    if directory == reference_dir:
        continue  # Skip the reference directory itself
    
    # Determine missing and extra files relative to the reference
    missing_files = reference_files - file_set
    extra_files = file_set - reference_files

    # Print out any discrepancies
    if missing_files:
        print(f"Missing files in {directory} (present in {reference_dir} but not here):")
        for file in missing_files:
            print(f"  {file}")
        print()

    if extra_files:
        print(f"Extra files in {directory} (not present in {reference_dir}):")
        for file in extra_files:
            print(f"  {file}")
        print()

if not any([missing_files, extra_files]):
    print("All directories contain the same files, ignoring extensions.")


In [ ]:
import os
import nibabel as nib

# Directory containing .nii.gz files
directory = 'D:\CTH_archive\CTP_STRIPPED'

# Iterate over all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a .nii.gz file
    if filename.endswith('.nii.gz'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)

        # Load the .nii.gz file
        nii_file = nib.load(file_path)

        # Construct the new file name with .nii extension
        new_file_path = file_path[:-3]  # Remove 'gz' from the end

        # Save the file as .nii
        nib.save(nii_file, new_file_path)

        # Optionally, remove the original .nii.gz file
        # os.remove(file_path)

        print(f'Converted {filename} to .nii format.')


In [ ]:
import os
import nibabel as nib
import shutil
import random
from tqdm import tqdm

num_test_files = 5


def convert_and_copy_with_labels_and_rename(image_source_dir, image_target_dir, label_source_dir, label_target_dir, images_test_dir, labels_test_dir):
    # Ensure all target directories exist, create if they don't
    for dir_path in [image_target_dir, label_target_dir, images_test_dir, labels_test_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    # List all .nii files in the source directory
    all_files = [f for f in sorted(os.listdir(image_source_dir)) if f.endswith('.nii')]
    
    # Determine the number of files to sample, ensuring it's not more than the total number of files available
    num_test_files = min(5, len(all_files))

    # Randomly select files for testing, based on the determined number
    test_files = random.sample(all_files, num_test_files)

    # Move selected test files and their labels
    for file_name in tqdm(test_files, desc="Moving test files"):
        shutil.move(os.path.join(image_source_dir, file_name), os.path.join(images_test_dir, file_name))
        
        # Adjust the file name for the label to match the .nii extension
        label_file_name = file_name.replace('.nii.gz', '.nii') if file_name.endswith('.nii.gz') else file_name
        shutil.move(os.path.join(label_source_dir, label_file_name), os.path.join(labels_test_dir, label_file_name))
        print(f"Moved {file_name} and its label to the test directories.")

    # Remove test files from the all_files list
    for test_file in test_files:
        all_files.remove(test_file)

    # Initialize a counter for unique naming within the training set
    counter = 1

    # Iterate over the remaining files for training
    for file_name in tqdm(all_files, desc="Processing training files"):
        # Construct full source file paths for images and labels
        image_source_file_path = os.path.join(image_source_dir, file_name)
        label_source_file_path = os.path.join(label_source_dir, file_name)

        # Load the .nii file (image)
        nii_image = nib.load(image_source_file_path)

        # Construct the new file name for images and labels
        base_name = file_name[:-4]  # Remove .nii extension
        new_image_name = f"{base_name}_{counter:03d}_0000.nii.gz"
        new_label_name = f"{base_name}_{counter:03d}.nii.gz"

        # Construct target file paths
        image_target_file_path = os.path.join(image_target_dir, new_image_name)
        label_target_file_path = os.path.join(label_target_dir, new_label_name)

        # Save the image file as .nii.gz in the target directory
        nib.save(nii_image, image_target_file_path)
        print(f"Converted and copied image: {image_source_file_path} to {image_target_file_path}")

        # Load the .nii file (label)
        nii_label = nib.load(label_source_file_path)

        # Save the label file as .nii.gz in the target directory
        nib.save(nii_label, label_target_file_path)
        print(f"Converted and copied label: {label_source_file_path} to {label_target_file_path}")

        # Increment the counter for unique naming
        counter += 1

# Define source and target directories for images and labels
image_source_dir = r"D:/CTH_archive/CTH_STRIPPED"
image_target_dir = r"D:/nnUNet_raw/Dataset039_Perfusion/imagesTr"
label_source_dir = r"D:/CTH_archive/TMAX_NIFTI_QUANT/REGISTERED_ADJUSTED"
label_target_dir = r"D:/nnUNet_raw/Dataset039_Perfusion/labelsTr"

# Define target directories for testing
images_test_dir = r"D:/nnUNet_raw/Dataset039_Perfusion/imagesTs"
labels_test_dir = r"D:/nnUNet_raw/Dataset039_Perfusion/labelsTs"

convert_and_copy_with_labels_and_rename(image_source_dir, image_target_dir, label_source_dir, label_target_dir, images_test_dir, labels_test_dir)



In [ ]:
import os
import json

def generate_dataset_json(dataset_dir, num_quant_levels, channel_names, nnUNet_dir, file_ending=".nii.gz", num_test_data=0):
    """
    Generate a dataset.json file for the given dataset with dynamic quantization levels.

    Args:
    - dataset_dir (str): Directory where the dataset files are stored.
    - num_quant_levels (int): Number of quantization levels (excluding the background).
    - channel_names (dict): Mapping of channel indices to their names.
    - file_ending (str): File extension of the dataset files.
    - num_test_data (int): The number of the dataset to be used for testing.

    Returns:
    - None
    """
    # Dynamically generate labels based on the number of quantization levels
    labels = {"background": "0"}
    for i in range(1, num_quant_levels + 1):
        labels[f"quantized_{i}"] = str(i)

    # Count the number of dataset files
    num_training = len([file for file in os.listdir(dataset_dir) if file.endswith(file_ending)])

    # Use the specified number of test data and calculate the remaining number of training files
    num_test = num_test_data

    # Construct the dataset JSON structure
    dataset_json = {
        "labels": labels,
        "numTraining": num_training,
        "numTest": num_test,
        "channel_names": channel_names,
        "file_ending": file_ending
    }

    # Write the JSON structure to a file
    with open(os.path.join(nnUNet_dir, "dataset.json"), 'w') as json_file:
        json.dump(dataset_json, json_file, indent=4)

    print(f"dataset.json file has been generated in {nnUNet_dir}")

# Example usage
dataset_dir = r'D:\nnUNet_raw\Dataset039_Perfusion\labelsTr'  # Path to dataset directory
nnUNet_dir = r'D:\nnUNet_raw\Dataset039_Perfusion'  # Path to nnUNet directory
num_quant_levels = 5  # Number of quantization levels (excluding background)
channel_names = {"0": "CT"}
num_test_data = 5  # Specify the number of test data

generate_dataset_json(dataset_dir, num_quant_levels, channel_names, nnUNet_dir, file_ending=".nii.gz", num_test_data=num_test_data)


In [ ]:
%matplotlib inline
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, Dropdown

prediction_dir = r"D:\CTH_archive\CTP_STRIPPED"
ground_truth_dir = r"D:\CTH_archive\TMAX_NIFTI_QUANT\REGISTERED"  # Updated path for ground truth images
ct_images_dir = r"D:\CTH_archive\CTH_NIFTI"  # Directory for CT head images

# Get the .nii.gz files in the directories
prediction_files = sorted([f for f in os.listdir(prediction_dir) if f.endswith('.nii') or f.endswith('.nii.gz')])
ground_truth_files = sorted([f for f in os.listdir(ground_truth_dir) if f.endswith('.nii') or f.endswith('.nii.gz')])
ct_image_files = sorted([f for f in os.listdir(ct_images_dir) if f.endswith('.nii') or f.endswith('.nii.gz')])

def apply_window(image, level=40, width=80):
    lower = level - (width / 2)
    upper = level + (width / 2)
    return np.clip((image - lower) / (upper - lower), 0, 1)

def plot_images(prediction_file, ground_truth_file, ct_image_file, slice_idx):
    # Load the files
    prediction_img = nib.load(os.path.join(prediction_dir, prediction_file))
    ground_truth_img = nib.load(os.path.join(ground_truth_dir, ground_truth_file))
    ct_img = nib.load(os.path.join(ct_images_dir, ct_image_file))

    # Convert the data to numpy arrays
    prediction_data = prediction_img.get_fdata()
    ground_truth_data = ground_truth_img.get_fdata()
    ct_data = ct_img.get_fdata()

    # Apply custom windowing to the CT head image
    ct_data_windowed = apply_window(ct_data)

    # Plot the ground truth image, the prediction, the windowed CT head image, and the windowed CT with ground truth overlay
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    axes[0].imshow(ground_truth_data[:, :, slice_idx], cmap='gray')
    axes[0].set_title('Ground Truth Image')
    axes[1].imshow(prediction_data[:, :, slice_idx], cmap='gray')
    axes[1].set_title('Prediction')
    axes[2].imshow(ct_data_windowed[:, :, slice_idx], cmap='gray')  # Show the windowed CT head image
    axes[2].set_title('Windowed CT Head Image')
    axes[3].imshow(ct_data_windowed[:, :, slice_idx], cmap='gray')  # Show the windowed CT head image again for overlay
    axes[3].imshow(ground_truth_data[:, :, slice_idx], cmap='hot', alpha=0.5)  # Overlay the ground truth
    axes[3].set_title('CT with Ground Truth Overlay')
    plt.show()


# Create widgets for file selection and slice index
prediction_file_widget = Dropdown(options=prediction_files)
ground_truth_file_widget = Dropdown(options=ground_truth_files)
ct_image_file_widget = Dropdown(options=ct_image_files)
slice_idx_widget = IntSlider(min=0, max=1, step=1, value=0)  # Initial max and value set to 0

# Create an interactive plot
plot = interactive(plot_images, prediction_file=prediction_file_widget, ground_truth_file=ground_truth_file_widget, ct_image_file=ct_image_file_widget, slice_idx=slice_idx_widget)

# Update the maximum value and value of the slice index slider whenever a new CT image is selected
def update_slice_idx_range(*args):
    ct_img = nib.load(os.path.join(ct_images_dir, ct_image_file_widget.value))
    ct_data = ct_img.get_fdata()
    slice_idx_widget.max = ct_data.shape[2] - 1
    slice_idx_widget.value = min(slice_idx_widget.value, slice_idx_widget.max)

ct_image_file_widget.observe(update_slice_idx_range, 'value')

# Manually call the update function to set the initial max value of the slice index slider
update_slice_idx_range()

# Display the interactive plot
display(plot)


In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, IntSlider, interactive

# Base directory paths
prediction_dir = r"D:\CTH_archive\CTP_STRIPPED"
ground_truth_dir = r"D:\CTH_archive\TMAX_NIFTI_QUANT\REGISTERED_ADJUSTED"
ct_images_dir = r"D:\CTH_archive\CTH_STRIPPED"

# Assuming all base filenames are the same across directories, we can just list files from one directory
file_options = sorted([f for f in os.listdir(ct_images_dir) if f.endswith('.nii') or f.endswith('.nii.gz')])

def apply_window(image, level=40, width=80):
    lower = level - (width / 2)
    upper = level + (width / 2)
    return np.clip((image - lower) / (upper - lower), 0, 1)

def plot_images(selected_file, slice_idx):
    # Construct file paths for each type of image using the selected file
    prediction_file_path = os.path.join(prediction_dir, selected_file)
    ground_truth_file_path = os.path.join(ground_truth_dir, selected_file)
    ct_image_file_path = os.path.join(ct_images_dir, selected_file)

    # Load the files
    prediction_img = nib.load(prediction_file_path)
    ground_truth_img = nib.load(ground_truth_file_path)
    ct_img = nib.load(ct_image_file_path)

    # Convert the data to numpy arrays
    prediction_data = prediction_img.get_fdata()
    ground_truth_data = ground_truth_img.get_fdata()
    ct_data = ct_img.get_fdata()

    # Apply custom windowing to the CT head image
    ct_data_windowed = apply_window(ct_data)

    # Plot the images
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    axes[0].imshow(ground_truth_data[:, :, slice_idx], cmap='gray')
    axes[1].imshow(prediction_data[:, :, slice_idx], cmap='gray')
    axes[2].imshow(ct_data_windowed[:, :, slice_idx], cmap='gray')
    axes[3].imshow(ct_data_windowed[:, :, slice_idx], cmap='gray')
    axes[3].imshow(ground_truth_data[:, :, slice_idx], cmap='hot', alpha=0.5)
    for ax in axes:
        ax.axis('off')
    plt.show()

# Create widgets for file selection and slice index
file_widget = Dropdown(options=file_options)
slice_idx_widget = IntSlider(min=0, max=1, step=1, value=0)

# Create an interactive plot
plot = interactive(plot_images, selected_file=file_widget, slice_idx=slice_idx_widget)

# Update the maximum value and value of the slice index slider whenever a new file is selected
def update_slice_idx_range(*args):
    ct_img = nib.load(os.path.join(ct_images_dir, file_widget.value))
    ct_data = ct_img.get_fdata()
    slice_idx_widget.max = ct_data.shape[2] - 1
    slice_idx_widget.value = min(slice_idx_widget.value, slice_idx_widget.max)

file_widget.observe(update_slice_idx_range, 'value')

# Manually call the update function to set the initial max value of the slice index slider
update_slice_idx_range()

# Display the interactive plot
display(plot)


In [ ]:
preds_list = [x.split(' ')[0] for x in os.listdir('D:\CTH_archive\TMAX_NIFTI_QUANT\REGISTERED')]
preds_list = [x.replace('.nii', '') for x in preds_list]
CTH_DICOM_SPLIT = [x.split(' ')[0] for x in os.listdir('D:\CTH_archive\CTH_DICOM')]


In [ ]:
names_of_interest = set(CTH_DICOM_SPLIT) - set(preds_list)

In [ ]:
directory_listing = os.listdir("D:\CTH_archive\CTH_DICOM")

In [ ]:
# Filtering process
extracted_paths = []
for file_name in directory_listing:
    for name in names_of_interest:
        if name in file_name:
            extracted_paths.append(file_name)
            break  # Stop the inner loop once a match is found

# Display the extracted file paths
for path in extracted_paths:
    print(path)

In [ ]:
import os

base_directory = "D:\\CTH_archive\\CTH_DICOM"

for path in extracted_paths:
    # Construct the full path for the patient directory
    patient_directory = os.path.join(base_directory, path)
    
    # Skip if the path is not a directory (e.g., it's a file like 'BILLIPS_JAMES.nii')
    if not os.path.isdir(patient_directory):
        continue

    # Extract the patient name from the path
    patient_name = path.split(' ')[0]  # Assuming patient name is the first part

    # Find the last subdirectory within the patient directory, which represents the latest session or date
    sessions = [d for d in os.listdir(patient_directory) if os.path.isdir(os.path.join(patient_directory, d))]
    sessions.sort()  # Sort to ensure order, in case it's based on date or sequence
    if not sessions:  # Check if the list is empty
        continue  # Skip if there are no session directories
    last_session = sessions[-1]  # Get the last session based on sorted order

    # Construct the full path for the last session directory
    session_directory = os.path.join(patient_directory, last_session)
    
    # Specify the output filename using the patient's name
    output_file = os.path.join("D:\\CTH_archive\\NIfTI_TEST_SET", f"{patient_name}.nii")
    
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Call the convert_series_to_nifti function for the last session directory
    convert_series_to_nifti(session_directory, output_file)


In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output

def load_dicom_series(directory):
    dicom_files = [(os.path.join(directory, f), f) for f in os.listdir(directory) if f.endswith('.dcm')]
    dicoms = [(pydicom.dcmread(path), file_name) for path, file_name in dicom_files]
    dicoms.sort(key=lambda x: int(x[0].InstanceNumber))
    return dicoms

def display_image(image, file_name):
    clear_output(wait=True)
    plt.imshow(image, cmap='gray')
    plt.title(file_name)  # Display the file name as the title
    plt.axis('off')
    plt.show()


def get_last_subdir(base_path):
    # List all the entries in the base path
    all_entries = os.listdir(base_path)
    
    # Filter out files, leaving only directories
    dir_entries = [entry for entry in all_entries if os.path.isdir(os.path.join(base_path, entry))]
    
    # Sort the directories to get the last one
    dir_entries.sort()
    
    # Return the full path of the last subdirectory if available, else return None
    return os.path.join(base_path, dir_entries[-1]) if dir_entries else None

# Example usage
base_path = r"D:\CTH_archive\CTH_DICOM\TOUATI_MOHAMED 3117292"
dicom_series_directory = get_last_subdir(base_path)

dicom_series = load_dicom_series(dicom_series_directory)

slider = widgets.IntSlider(value=0, min=0, max=len(dicom_series)-1, step=1, description='Image Index:')

def update_image(image_index):
    dicom_data, file_name = dicom_series[image_index]
    display_image(dicom_data.pixel_array, file_name)

widgets.interactive(update_image, image_index=slider)
